### Find and recognize unknown faces from known people faces using face_recognition API ( 1 to many faces)

In [4]:
import face_recognition as fr
import numpy as np
import pandas as pd
from PIL import Image,ImageEnhance #, ImageDraw
from imutils.face_utils import FaceAligner
from imutils.face_utils import rect_to_bb
import imutils
import os
import cv2
import dlib
import time
import warnings
warnings.filterwarnings("ignore")
from pandas import ExcelWriter

In [5]:
CASE_PATH                   = "C:/Users/sriha/Documents/My_Data/Work_Documents/face_detection_emotion/Data/haarcascade_frontalface_default.xml"
path                        = "C:/Users/sriha/Documents/My_Data/Work_Documents/face_rec/FaceLandmarks/"



os.chdir(path)

################## Face detectors

hog_face_detector           = dlib.get_frontal_face_detector()
face_classifier             = cv2.CascadeClassifier(CASE_PATH)

predictor                   = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")
fa                          = FaceAligner(predictor, desiredFaceWidth=256)

known_face_encodings        = []
known_face_names            = []
known_face_encode_error     = []

os.chdir(path + "train/")

### Routine to capture basic 128 measurements (known as embedding) for each known face image

In [6]:
def known_image_encode(img,Upsample,knownimage):
    
    # Preprocessing steps
    # Resizing the image
    # Convert to grayscale
    # Apply CLAHE (Contrast Limited Adaptive Histogram Equalization) to improve the contrast and smooth out lighting differences
    # Apply Bilateral Filter to smoothening and reducing noise, while preserving edges

    img   = imutils.resize(img, width=400)
    gray  = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(5,8)) # Divide image into small blocks called “tiles” (Size is 8x8 by default in OpenCV)
    gray  = clahe.apply(gray)
    #res   = np.hstack((gray,clahe)) #stacking images side-by-side
    gray  = cv2.bilateralFilter(gray,5,75,75) # bilateral filter d = 15 and sigmaColor = sigmaSpace = 75. The greater its value, the more further pixels will mix together, given that their colors lie within the sigmaColor range
    rects = hog_face_detector(gray, Upsample) # Upsampling each image to detect faces
    if (len(rects) > 0):

        for rect in rects:
            faceAligned   = fa.align(img, gray, rect)
            try:
                face_encoding = fr.face_encodings(faceAligned)[0]  # Generate basic measurements
                known_face_encodings.append(face_encoding)
                known_face_names.append(knownimage)
            except Exception as e:
                known_face_encode_error.append(knownimage)
            break
    else:
        known_face_encode_error.append(knownimage)

    return()

#### Load all known images from drive to get basic 128 measurements with upsample = 1

In [7]:
known_face_encodings = []
known_face_names     = []
#image_load_error     = []
Upsample             = 1
starttime            = time.time()

for dirpath, dnames, knownimages in os.walk(path +"train/"):
    for knownimage in knownimages:
        img   = fr.load_image_file(path + "train/" + knownimage)
        known_image_encode(img,Upsample,knownimage)
       
print("Processed {0}% (i.e) {1} known faces completed capturing basic 128 facial measurements. Error in capturing {2} known faces".format(str(round(len(known_face_encodings)/(len(known_face_encodings) + len(known_face_encode_error)) * 100)),len(known_face_encodings),len(known_face_encode_error)))
print('Time taken = {} Min'.format(round((time.time() - starttime)/60)))

Processed 90% (i.e) 95 known faces completed capturing basic 128 facial measurements. Error in capturing 10 known faces
Time taken = 1 Min


#### Iterate capturing basic facial measurements for failed images  by incrementing the Upsample. Repeat upto 4 iterations

In [8]:
# Repeat for 3 iterations. Upsampling the image helps to detect smaller faces
repeat = 1

while ( (len(known_face_encode_error) > 0) and (repeat <=3)):
    
    known_face_encode_error_bk  = known_face_encode_error
    known_face_encode_error     = []
    Upsample                    = Upsample + 1
    starttime                   = time.time()
    
    for i in range(len(known_face_encode_error_bk)):
        img   = fr.load_image_file(path + "train/" + known_face_encode_error_bk[i])
        known_image_encode(img,Upsample,known_face_encode_error_bk[i])
    
    print("\nProcessed {0}% (i.e) {1} known faces completed capturing basic 128 facial measurements. Error in capturing {2} known faces".format(str(round(len(known_face_encodings)/(len(known_face_encodings) + len(known_face_encode_error)) * 100)),len(known_face_encodings),len(known_face_encode_error)))
    print('Time taken = {} seconds'.format(round(time.time() - starttime)))
    repeat += 1


Processed 91% (i.e) 96 known faces completed capturing basic 128 facial measurements. Error in capturing 9 known faces
Time taken = 3 seconds

Processed 92% (i.e) 97 known faces completed capturing basic 128 facial measurements. Error in capturing 8 known faces
Time taken = 9 seconds

Processed 93% (i.e) 98 known faces completed capturing basic 128 facial measurements. Error in capturing 7 known faces
Time taken = 30 seconds


### List the failed known image list (Unable to capture basic 128 measurements)

In [9]:
known_face_encode_error

['train13.jpg',
 'train26.jpg',
 'train52.jpg',
 'train59.jpg',
 'train66.jpg',
 'train72.jpg',
 'train99.jpg']

In [10]:
# Skip test images of corresponding failed known images which were unable to encode

for k in range(len(known_face_encode_error)):
    known_face_encode_error[k] = known_face_encode_error[k].replace('train', 'test')

In [11]:
def check_availability(element, collection: iter):
    return element in collection

#### Routine to compare each unknown image with known face encodings and the closest measurement based on Euclidean distance

In [12]:
# Check is unknown face is a match in the known face(s)
# if distance between unknown and known face is <= tolerance is a match (0.6)

def compare_Unknown(testImage,Upsample):
   
    unknown_image      = fr.load_image_file(path + 'test/'+ testImage)
    name               = "NoMatch"
    
    # resize, convert and align
    unknown_image   = imutils.resize(unknown_image, width=400)
    gray            = cv2.cvtColor(unknown_image, cv2.COLOR_BGR2GRAY) 
    clahe           = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(5,8)) # Divide image into small blocks called “tiles” (Size is 8x8 by default in OpenCV)
    gray            = clahe.apply(gray)
    gray            = cv2.bilateralFilter(gray,5,75,75) # bilateral filter d = 15 and sigmaColor = sigmaSpace = 75. The greater its value, the more further pixels will mix together, given that their colors lie within the sigmaColor range
    rects           = hog_face_detector(gray, Upsample)
    
    for rect in rects:
        unknown_image          = fa.align(unknown_image, gray, rect)
        face_locations         = fr.face_locations(unknown_image,model="hog")   
        #face_locations         = fr.face_locations(unknown_image,model="cnn")   
        unknown_face_encodings = fr.face_encodings(unknown_image, face_locations) # Encoding one face since each image only has one face
        
    for (top, right, bottom, left), unknown_face_encoding in zip(face_locations, unknown_face_encodings):
     
        matches          = fr.compare_faces(known_face_encodings, unknown_face_encoding,tolerance=0.6)
        distance         = fr.face_distance(known_face_encodings, unknown_face_encoding)
        distance         = list(np.around(np.array(distance),2))
        best_match_index = np.argmin(distance)   # Get smallest distance between the two faces
        min_distance     = distance[best_match_index]
        
        if matches[best_match_index]:
            result           = 'Match '
            Known_face_match = known_face_names[best_match_index]   
        else:
            result            = "NoMatch"
            Known_face_match  = "No_Known_Face"

    return result,Known_face_match,min_distance 

#### For each unknown image, extract basic 128 measurements with upsample = 1. Then Compare each unknown image measurements against known faces measurements captured and get close match image distance

In [13]:
cols                      = ['UnknownFace', 'Knownface','Distance','Predicted']
lst                       = []
Unknown_face_encode_error = []
skip_Unknown_face         = []  
Upsample                  = 1
starttime                 = time.time()

for dirpath, dnames, unknownimages in os.walk(path + 'test/'):
    for unknownimage in unknownimages:
        if not (check_availability(unknownimage, known_face_encode_error)): # Skip erroroneous unkown faces
            try:
                result,knownface,dist = compare_Unknown(unknownimage,Upsample)
                lst.append([unknownimage,knownface,dist,result])
                
            except Exception as e:
                Unknown_face_encode_error.append(unknownimage)
                continue
        else:
            skip_Unknown_face.append(unknownimage)
            
output  = pd.DataFrame(lst, columns=cols)
print("\nProcessed {0} Unknown faces. Error in comparing {1} unknown faces".format(len(output),len(Unknown_face_encode_error)))
print('Time taken = {} seconds'.format(round(time.time() - starttime)))


Processed 92 Unknown faces. Error in comparing 7 unknown faces
Time taken = 42 seconds


#### For each erroneous unknown face, try extract basic 128 measurements by incrementing upsample by 1. Then Compare each unknown person measurements against known faces measurements captured and get close match image distance

In [14]:
# Repeat for 3 iterations

repeat = 1
while ( (len(Unknown_face_encode_error) > 0) and (repeat <=3)):
    
    Unknown_face_encode_error_bk = Unknown_face_encode_error
    Unknown_face_encode_error    = []
    Upsample                     = Upsample + 1
    starttime                    = time.time()
    print('\nUpsample =',Upsample)
           
    for i in range(len(Unknown_face_encode_error_bk)):
        
        if not (check_availability(Unknown_face_encode_error_bk[i],known_face_encode_error)): # Skip erroroneous unkown faces
            try:
                result,knownface,dist = compare_Unknown(Unknown_face_encode_error_bk[i],Upsample)
                lst.append([unknownimage,knownface,dist,result])
                
            except Exception as e:
                Unknown_face_encode_error.append(Unknown_face_encode_error_bk[i])
                continue
        else:
            skip_Unknown_face.append(Unknown_face_encode_error_bk[i])
            
    output  = pd.DataFrame(lst, columns=cols)
    print("Processed {0} Unknown faces. Error in comparing {1} unknown faces".format(len(output),len(Unknown_face_encode_error)))
    print('Time taken = {} seconds'.format(round(time.time() - starttime)))
    repeat += 1


Upsample = 2
Processed 95 Unknown faces. Error in comparing 4 unknown faces
Time taken = 3 seconds

Upsample = 3
Processed 97 Unknown faces. Error in comparing 2 unknown faces
Time taken = 5 seconds

Upsample = 4
Processed 97 Unknown faces. Error in comparing 2 unknown faces
Time taken = 8 seconds


### Predicted Unknown Face comparasion match results

In [19]:
output.head(15)

,UnknownFace,Knownface,Distance,Predicted
0,IMG_0040.jpg,Srihari.jpg,0.51,Match
1,test1.jpg,train1.jpg,0.45,Match
2,test10.jpg,train10.jpg,0.53,Match
3,test100.jpg,train46.jpg,0.50,Match
4,test101.jpg,train101.jpg,0.34,Match
5,test102.jpg,train28.jpg,0.52,Match
6,test11.jpg,train11.jpg,0.49,Match
7,test12.jpg,train12.jpg,0.48,Match
8,test14.jpg,train14.jpg,0.41,Match
9,test15.jpg,train15.jpg,0.46,Match


### Get the actual names of unknown images and calculate the accuracy

In [46]:
New = Old = Revised = []
Match = []
New = output.UnknownFace
matchCount = nomatchCount = 0

for k in range(len(New)):
    Revised.append(New[k].replace('test', 'train'))

New   = Revised
Old   = output.Knownface

# Compare known and unknown names

for k in range(len(New)):
    
    if ( (Old[k] == 'No_Known_Face')):
        Match.append('NoMatch')
        nomatchCount += 1
        
    elif (New[k] == Old[k]):
        Match.append('Match')
        matchCount += 1

    elif (New[k] != Old[k]):
        
        if (Old[k] == 'Srihari.jpg'):
            Match.append('Match')
            matchCount += 1
        else:
            Match.append('NoMatch')
            nomatchCount += 1
            
output['Actual'] = Match

print(" From total images of {0}, there were {1} actual matches and {2} No matches".format(len(output),matchCount, nomatchCount))
print(" Accuracy : {0}".format(round(matchCount/len(output),2)))


 From total images of 97, there were 62 actual matches and 35 No matches
 Accuracy : 0.64


### Face Comparasion Results

In [47]:
output.head(20)

,UnknownFace,Knownface,Distance,Predicted,Actual
0,IMG_0040.jpg,Srihari.jpg,0.51,Match,Match
1,test1.jpg,train1.jpg,0.45,Match,Match
2,test10.jpg,train10.jpg,0.53,Match,Match
3,test100.jpg,train46.jpg,0.50,Match,NoMatch
4,test101.jpg,train101.jpg,0.34,Match,Match
5,test102.jpg,train28.jpg,0.52,Match,NoMatch
6,test11.jpg,train11.jpg,0.49,Match,Match
7,test12.jpg,train12.jpg,0.48,Match,Match
8,test14.jpg,train14.jpg,0.41,Match,Match
9,test15.jpg,train15.jpg,0.46,Match,Match


### Append the failed and skipped unknown image details

In [48]:
for i in range(len(Unknown_face_encode_error)):
    new_row = {'UnknownFace':Unknown_face_encode_error[i], 'Knownface':'Error extracting basic measurements', 'Distance':'','Predicted':'','Actual':''}       
    output = output.append(new_row, ignore_index=True)
        
for i in range(len(known_face_encode_error)):
    new_row = {'UnknownFace':known_face_encode_error[i], 'Knownface':'Skip', 'Distance':'','Predicted':'','Actual':''}       
    output = output.append(new_row, ignore_index=True)
        

In [49]:
output.head(20)

,UnknownFace,Knownface,Distance,Predicted,Actual
0,IMG_0040.jpg,Srihari.jpg,0.51,Match,Match
1,test1.jpg,train1.jpg,0.45,Match,Match
2,test10.jpg,train10.jpg,0.53,Match,Match
3,test100.jpg,train46.jpg,0.5,Match,NoMatch
4,test101.jpg,train101.jpg,0.34,Match,Match
5,test102.jpg,train28.jpg,0.52,Match,NoMatch
6,test11.jpg,train11.jpg,0.49,Match,Match
7,test12.jpg,train12.jpg,0.48,Match,Match
8,test14.jpg,train14.jpg,0.41,Match,Match
9,test15.jpg,train15.jpg,0.46,Match,Match


### Write comparasion macth results to Excel File for verification

In [ ]:
writer = ExcelWriter('C:/Users/sriha/Desktop/faceCompare_Stats.xlsx')
output.to_excel(writer)
writer.save()